# Running V1 Model from Cybershuttle

Objective - Show how the Cybershuttle SDK enables to run multiple V1 simulations from a Jupyter Notebook.

## Step 1 - Import SDK, Authenticate, and Find Runtimes

In [ ]:
import airavata_experiments as ae
import airavata_experiments.neuro as ns

# log in to cybershuttle
ae.login("api.cybershuttle.org")

# find what runtimes (on the anvil cluster) can provide 100 cpus for 1 hr
runtimes_with_hi_cpu = ae.list_runtimes(
  cluster="anvil.rcac.purdue.edu",
  category="cpu",
  group="Default",
  walltime=60,
  cpu_count=16,
)

In [ ]:
# Run the first V1 simulation
exp1 = ns.V1.initialize(
  name="plan_1_1000",
  lgn_spikes=ns.SpikeInput.from_h5("spikes_lgn.trial_0.h5", node_set="lgn"),
  bkg_spikes=ns.SpikeInput.from_h5("spikes_bkg.trial_0.h5", node_set="bkg"),
  injcur_spikes=None,
  duration=1000.0,
  dt=0.25
)
exp1.create_task(*runtimes_with_hi_cpu)
plan1 = exp1.plan()
plan1.launch()

In [ ]:
# Run another V1 simulation (in parallel)
exp2 = ns.V1.initialize(
  name="plan_2_3000",
  lgn_spikes=ns.SpikeInput.from_h5("spikes_lgn.trial_0.h5", node_set="lgn"),
  bkg_spikes=ns.SpikeInput.from_h5("spikes_bkg.trial_0.h5", node_set="bkg"),
  injcur_spikes=None,
  duration=3000.0,
  dt=0.25
)
exp2.create_task(*runtimes_with_hi_cpu)
plan2 = exp2.plan()
plan2.launch()

In [ ]:
# Wait for both plans to complete
plan1.wait_for_completion()
plan2.wait_for_completion()

In [ ]:
# Download the results of both plans
plan1.download(f"results/${exp1.name}")
plan2.download(f"results/${exp2.name}")